In [2]:
# !pip install selenium
# !pip install selenium chromedriver-autoinstaller
# !pip install webdriver-manager
# !pip install undetected-chromedriver
# !pip install scikit-image
# !pip install user_agent
# !pip install pyautogui
# !pip install fake_useragent
# !pip install selenium-stealth

In [16]:
# !apt-get update
# !apt-get install -y chromium-browser
# !apt-get install chromium-chromedriver


  Using cached undetected_chromedriver-3.5.5-py3-none-any.whl
  Attempting uninstall: undetected-chromedriver
    Found existing installation: undetected-chromedriver 3.0.0
    Uninstalling undetected-chromedriver-3.0.0:
      Successfully uninstalled undetected-chromedriver-3.0.0


In [1]:
import os
import random
import requests
import time
import user_agent
import pyautogui
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Define download directory within the 'Copilot' folder
download_dir = os.path.join(current_dir, 'Copilot')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Define Chrome options and experimental preferences
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-popup-blocking")

prefs = {
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.pattern_pairs.*.multiple-automatic-downloads": 1,
    "download.prompt_for_download": False,
    "download.default_directory": download_dir
}
options.add_experimental_option("prefs", prefs)

try:
    # Use webdriver_manager to automatically download and manage ChromeDriver
    driver_path = ChromeDriverManager().install()
    driver = uc.Chrome(options=options)
    driver.get("https://www.bing.com/images/create?FORM=IRPGEN")
except WebDriverException as e:
    print(f"An error occurred while initializing the WebDriver: {e}")


In [3]:
# Open new tabs in the existing window with Bing Image Search for creating images
for i in range(1):
    driver.execute_script("window.open('https://www.bing.com/images/create?FORM=IRPGEN', '_blank');")

In [4]:
current_dir = os.getcwd()

# Get screen dimensions for random movement within the screen
screen_width, screen_height = pyautogui.size()

# Randomize mouse movement within the screen
x = random.randint(0, screen_width)
y = random.randint(0, screen_height)

# Define download directory within the Copilot folder
download_dir = os.path.join(current_dir, 'Copilot')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Read prompts from prompts.txt file
with open('prompts.txt', 'r') as file:
    prompts = file.readlines()
    num_prompts = len(prompts)
    

    # Initialize a counter for prompt number
    prompt_number = 568

    # Loop through each set of prompts for each tab
    for i in range(0, num_prompts, len(driver.window_handles)):
        # Loop through each tab and give the prompt from the current set
        for tab_index in range(len(driver.window_handles)):
            try:
                # Get the prompt text for the current tab index
                prompt_text = prompts[prompt_number + tab_index-1].strip()
                while prompt_text == "-":
                    prompt_number+= 1
                    prompt_text=prompts[(prompt_number + tab_index - 1) % len(prompts)].strip()

                driver.switch_to.window(driver.window_handles[tab_index])

                # Find the text input field using its name attribute
                time.sleep(random.uniform(5, 10))
                text_input = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.NAME, "q")))
                # Clear any existing text in the input field
                text_input.clear()
                # Enter the prompt text into the text input field
                text_input.send_keys(prompt_text)

                pyautogui.moveTo(x, y, duration=random.uniform(0.5, 1))

                # Find the div containing the 'Create' button and click on it
                time.sleep(random.uniform(2, 5))
                create_button = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.ID, "create_btn_i")))
                create_button.click()
                time.sleep(random.uniform(1, 2.5))

            except Exception as e:
                try:
                    time.sleep(random.uniform(3, 5))
                    driver.get("https://www.bing.com/images/create?FORM=IRPGEN")
                    text_input = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.NAME, "q")))
                    # Clear any existing text in the input field
                    text_input.clear()
                    # Enter the prompt text into the text input field
                    text_input.send_keys(prompt_text)
                except Exception as e:
                    driver.get("https://www.bing.com/images/create?FORM=IRPGEN")
                    print(f"Exception occurred in tab {tab_index+1}: Create button not clicked")
                time.sleep(random.uniform(2, 5))
                continue

        # Loop to get the image source URLs and download images
        for tab_index in range(len(driver.window_handles)):
            driver.switch_to.window(driver.window_handles[tab_index])

            # Wait for a specified time only in the first tab
            if tab_index == 0:
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(random.uniform(60, 80))

            try:
                # Wait for a random time
                driver.get(driver.current_url)
                time.sleep(random.uniform(2, 4))

                # Wait for the image element to be visible
                image_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//img[@class='mimg']")))

                # Get the image source URL
                image_src = image_element.get_attribute("src")

                # Download the image using requests
                response = requests.get(image_src)

                # Save the image to a file with the prompt number as part of the name
                image_name = f"Tweet_{prompt_number}.jpg"
                with open(os.path.join(download_dir, image_name), 'wb') as image_file:
                    image_file.write(response.content)
                    print(f"{image_name} downloaded:")
                driver.get("https://www.bing.com/images/create?FORM=IRPGEN")
                time.sleep(random.uniform(1, 2.5))

            except Exception as e:
                print(f"Image not downloaded for {prompt_number} prompt in tab {tab_index + 1}")
                driver.get("https://www.bing.com/images/create?FORM=IRPGEN")
                time.sleep(random.uniform(1, 2.5))

            prompt_number += 1

Tweet_568.jpg downloaded:
Tweet_569.jpg downloaded:


In [20]:
print(f"{7622- 7555}")

67


In [8]:
driver.refresh()